# Abstract

In [57]:
#importing libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [58]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1                        # Number of parallel threads used to run Algorithms
min_mem_size=6 
classification= True
scale=False
max_models=None    
model_path=None
balance_y=False 
balance_threshold=0.2             
name=None 
server_path=None  
analysis=0 

When enabled, H2O will either undersample the majority classes or oversample the minority classes. Note that the resulting model will also correct the final probabilities (“undo the sampling”) using a monotonic transform, so the predicted probabilities of the first model will differ from a second model. However, because AUC only cares about ordering, it won’t be affected.

In [59]:
data_path='C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv'

### Data Cleaning

In [60]:
import pandas as pd
data= pd.read_csv("Employee_access_data.csv", decimal = ',')

In [61]:
data.dtypes

ACTION              int64
RESOURCE            int64
MGR_ID              int64
ROLE_ROLLUP_1       int64
ROLE_ROLLUP_2       int64
ROLE_DEPTNAME       int64
ROLE_TITLE          int64
ROLE_FAMILY_DESC    int64
ROLE_FAMILY         int64
ROLE_CODE           int64
dtype: object

**Checking for the Null values**

In [62]:
total = data.isnull().sum()[data.isnull().sum() != 0].sort_values(ascending = False)
percent = pd.Series(round(total/len(data)*100,2))
pd.concat([total, percent], axis=1, keys=['total_missing', 'percent'])

,total_missing,percent


No null values

### RUNTIME

300, 500, 700, 1000, 1200

In [63]:
run_time= 700

In [64]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size, "GB")

2 GB


In [65]:
#Defining functions

#generating random run_id
def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str

# storing in m_data dictionary  
def set_meta_data(analysis,run_id,server,data,model_path,run_time,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  # m_data['target']=target
  #m_data['classification']=classification
  m_data['project'] =name
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['start_time_sec'] = time.time()
  m_data['min_mem_size'] = min_mem_size
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold 
  m_data['max_models']=model
  m_data['scale']=scale  
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  
  m_data['analysis'] = analysis
  m_data['end_time_sec'] = time.time()  
  return m_data

#converting dictionary to json
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
# Segregating in different lists of int, enum, reals and checking for missing values and then scaling(standardizing)    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return

# Determining Independent variables (X) from the dataset 
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

#Checkinh if X does not exist then remove
def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    

### RUN_ID

In [66]:
#randomly generating run_id through alphabet function
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
a = run_id + '_EmpAccess_' + str(run_time)
run_dir = os.path.join(server_path,a)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id) 

KYUjJqIuc


In [67]:
# # Creating a log file
# logfile=run_id+'_autoh2o_log.zip'
# logs_path=os.path.join(run_dir,'logs')
# print(logs_path,' ',logfile)

In [68]:
name = run_id+'_EmpAccess_' + str(run_time)
name

'KYUjJqIuc_EmpAccess_700'

In [69]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,model_path,run_time,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)  

{'project': 'KYUjJqIuc_EmpAccess_700', 'run_time': 700, 'run_id': 'KYUjJqIuc', 'start_time_sec': 1555606780.2417612, 'min_mem_size': 2, 'balance': False, 'balance_threshold': 0.2, 'max_models': None, 'scale': False, 'model_path': None, 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project', 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv', 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\KYUjJqIuc_EmpAccess_700', 'nthreads': 1, 'analysis': 0, 'end_time_sec': 1555606780.2417612}


In [70]:
if classification :
    meta_data["Problem_type"] = "Classification"
    print("Problem Type:  Classification")
else:
    meta_data["Problem_type"] = "Regression"
    print("Problem Type:  Regression")

Problem Type:  Classification


In [71]:
# 65535 Highest port no
port_no=random.randint(5555,55555)

#initializing H2O
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:43243 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from C:\Users\hp\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\hp\AppData\Local\Temp\tmp5dsqdm3b
  JVM stdout: C:\Users\hp\AppData\Local\Temp\tmp5dsqdm3b\h2o_hp_started_from_python.out
  JVM stderr: C:\Users\hp\AppData\Local\Temp\tmp5dsqdm3b\h2o_hp_started_from_python.err
  Server is running at http://127.0.0.1:43243
Connecting to H2O server at http://127.0.0.1:43243 ... successful.


H2O cluster uptime:,05 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_hp_668n1t
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [72]:
print(data_path)              #path of csv 

C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv


In [73]:
#importing data file on h2o server
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [74]:
df.head()

ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
1,36724,14457,118219,118220,117884,117879,267952,19721,117880
1,36135,5396,117961,118343,119993,118321,240983,290919,118322
1,42680,5905,117929,117930,119569,119323,123932,19793,119325
0,45333,14561,117951,117952,118008,118568,118568,19721,118570
1,25993,17227,117961,118343,123476,118980,301534,118295,118982
1,19666,4209,117961,117969,118910,126820,269034,118638,126822
1,31246,783,117961,118413,120584,128230,302830,4673,128231
1,78766,56683,118079,118080,117878,117879,304519,19721,117880


In [75]:
rows = len(df)
print("Total rows in the data set = ", rows)
cols = len(df.columns)
print("Total Columns in the data set = ", cols)

rowscols = df.shape
shape = rows * cols
print("rows X columns = ", rowscols)
print("Total Records = ", shape)

meta_data['Total Records'] = shape

Total rows in the data set =  32769
Total Columns in the data set =  10
rows X columns =  (32769, 10)
Total Records =  327690


In [76]:
df.describe()

Rows:32769
Cols:10




,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
type,int,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,25.0,4292.0,23779.0,4674.0,117879.0,4673.0,3130.0,117880.0
mean,0.9421099209618847,42923.916170771125,25988.957978577324,116952.62778845867,118301.82315603165,118912.77991394309,125916.15264426745,170178.36964814307,183703.40889255097,119789.43013213709
maxs,1.0,312153.0,311696.0,311178.0,286791.0,286792.0,311867.0,311867.0,308574.0,270691.0
sigma,0.23353903780676308,34173.892702138255,35928.03165014073,10875.563591093745,4551.588572012568,18961.32291708769,31036.465824743256,69509.46213013002,100488.40741337684,5784.275515531029
zeros,1897,13,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0
0,1.0,39353.0,85475.0,117961.0,118300.0,123472.0,117905.0,117906.0,290919.0,117908.0
1,1.0,17183.0,1540.0,117961.0,118343.0,123125.0,118536.0,118536.0,308574.0,118539.0
2,1.0,36724.0,14457.0,118219.0,118220.0,117884.0,117879.0,267952.0,19721.0,117880.0


In [77]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

### Depentdent Variable

In [78]:
# dependent variable
# assign target an d inputs for classification or regression
if target==None:
    target="ACTION"   
y = target
meta_data['Target']=y
y

'ACTION'

### Independent Variables

In [79]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [80]:
# independent variables
# putting all independent variables in the list X 

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
  print(X)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables to meta data

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']


### Problem Type

The dependent variable for this data set is of Binary/ classification type.

In [81]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [82]:
df[y]

ACTION
1
1
1
1
1
0
1
1
1
1


**Checking the number of classes of dependent variable**

In [83]:
# Total categories in the target column
lvl = df[y].levels()
print(lvl)
meta_data["levels"] = lvl

[['0', '1']]


y has 2 levels or classes

In [84]:
# checking value of target for real, int, enum
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok   

In [85]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [86]:
# since y is enum type
print(ok)

True


In [87]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    

In [88]:
# getting the data types of all the variables in the dataset
allV=get_variables_types(df)
allV

{'ACTION': 'enum',
 'RESOURCE': 'int',
 'MGR_ID': 'int',
 'ROLE_ROLLUP_1': 'int',
 'ROLE_ROLLUP_2': 'int',
 'ROLE_DEPTNAME': 'int',
 'ROLE_TITLE': 'int',
 'ROLE_FAMILY_DESC': 'int',
 'ROLE_FAMILY': 'int',
 'ROLE_CODE': 'int'}

In [89]:
# Adding the data types to meta data 
meta_data['variables']=allV

### Using H2OAutoML

* Used for automatic training and tuning of the models

In [90]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [91]:
model_start_time = time.time()

In [92]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [93]:
execution_time =  time.time() - model_start_time
meta_data['model_execution_time_sec'] = execution_time
print("Execution time for ", run_time,"sec =  ",meta_data['model_execution_time_sec'])

Execution time for  700 sec =   708.7163550853729


## LeaderBoard

In [94]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [95]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_grid_1_AutoML_20190418_130009_model_1,0.867092,0.152078,0.303534,0.200599,0.040240
1,StackedEnsemble_BestOfFamily_AutoML_20190418_1...,0.863712,0.154195,0.306007,0.198776,0.039512
2,StackedEnsemble_AllModels_AutoML_20190418_130009,0.862187,0.154182,0.306814,0.198965,0.039587
3,DRF_1_AutoML_20190418_130009,0.845773,0.177915,0.363560,0.201498,0.040602
4,GBM_4_AutoML_20190418_130009,0.841592,0.159551,0.364933,0.202601,0.041047
5,XRT_1_AutoML_20190418_130009,0.840561,0.188540,0.356365,0.202947,0.041188
6,GBM_3_AutoML_20190418_130009,0.839477,0.162258,0.385753,0.205124,0.042076
7,GBM_5_AutoML_20190418_130009,0.837055,0.165108,0.409014,0.207468,0.043043
8,GBM_2_AutoML_20190418_130009,0.834602,0.164862,0.405948,0.206954,0.042830
9,GBM_1_AutoML_20190418_130009,0.830971,0.166941,0.372203,0.208379,0.043422


In [96]:
length = len(aml_leaderboard_df)- 1
length
meta_data["Models_generated"] = length

### Saving Leaderboard into csv

In [97]:
# save leaderboard to csv
# run_time = run_time.ascharacter()
leaderboard_stats=run_id+'EmpAccess_'+ str(run_time) + '_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

### Generating hyperparameters for the best model on the leaderboard

In [98]:
# STart best model as first model
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [99]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_grid_1_AutoML_20190418_130009_model_1',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_grid_1_AutoML_20190418_130009_model_1'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_py_13_sid_a596',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_py_13_sid_a596'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 

In [100]:
# generating normalized coeff.
mods=mod_best.coef_norm
print(mods)


Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20190418_130009_model_1


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.020898010897479797
RMSE: 0.14456144332940163
LogLoss: 0.08166822239808458
Mean Per-Class Error: 0.026034879953249268
AUC: 0.9959055521033129
pr_auc: 0.9973188350179899
Gini: 0.9918111042066258
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7195748209356244: 


,0,1,Error,Rate
0,1565.0,332.0,0.175,(332.0/1897.0)
1,204.0,30668.0,0.0066,(204.0/30872.0)
Total,1769.0,31000.0,0.0164,(536.0/32769.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7195748,0.9913370,223.0
max f2,0.6119304,0.9945850,263.0
max f0point5,0.8267385,0.9935334,180.0
max accuracy,0.7708024,0.9836736,205.0
max precision,0.9938092,1.0,0.0
max recall,0.3868958,1.0,339.0
max specificity,0.9938092,1.0,0.0
max absolute_mcc,0.7848756,0.8576590,199.0
max min_per_class_accuracy,0.8602532,0.9720611,164.0
max mean_per_class_accuracy,0.8778646,0.9739651,154.0


Gains/Lift Table: Avg response rate: 94.21 %, avg score: 94.20 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-04-18 13:06:26,1 min 17.471 sec,0.0,0.2335355,0.2211221,0.5,0.0,1.0,0.0578901
,2019-04-18 13:06:26,1 min 17.706 sec,5.0,0.2307493,0.2104724,0.9624352,0.9121880,1.0614473,0.0415026
,2019-04-18 13:06:27,1 min 17.971 sec,10.0,0.2282909,0.2025158,0.9732929,0.9826041,1.0614473,0.0381458
,2019-04-18 13:06:27,1 min 18.190 sec,15.0,0.2262511,0.1967308,0.9708708,0.9947359,1.0614473,0.0385120
,2019-04-18 13:06:27,1 min 18.440 sec,20.0,0.2235849,0.1898370,0.9735035,0.9964851,1.0614473,0.0376881
---,---,---,---,---,---,---,---,---,---
,2019-04-18 13:06:38,1 min 29.337 sec,235.0,0.1482957,0.0853040,0.9956264,0.9969129,1.0614473,0.0168147
,2019-04-18 13:06:38,1 min 29.603 sec,240.0,0.1471853,0.0842346,0.9957301,0.9982797,1.0614473,0.0164790
,2019-04-18 13:06:39,1 min 29.875 sec,245.0,0.1460697,0.0831650,0.9958457,0.9977362,1.0614473,0.0163569
,2019-04-18 13:06:39,1 min 30.125 sec,250.0,0.1450561,0.0821668,0.9958806,0.9975440,1.0614473,0.0162959



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
MGR_ID,9084.5351562,1.0,0.2468362
RESOURCE,6941.0834961,0.7640549,0.1885964
ROLE_FAMILY_DESC,5498.6826172,0.6052795,0.1494049
ROLE_DEPTNAME,4547.2573242,0.5005493,0.1235537
ROLE_ROLLUP_2,2876.4724121,0.3166340,0.0781567
ROLE_TITLE,2615.2917480,0.2878839,0.0710602
ROLE_CODE,2528.9309082,0.2783776,0.0687137
ROLE_ROLLUP_1,1393.8673096,0.1534330,0.0378728
ROLE_FAMILY,1317.7751465,0.1450570,0.0358053


<bound method ModelBase.coef_norm of >


## Generating and storing Hyperparameters of each model into JSON

In [101]:
model_set=aml_leaderboard_df['model_id']
type(model_set)

pandas.core.series.Series

In [102]:
jsonDicts = []
for m in model_set.iteritems():
    m,model_name = m
    mod_best = h2o.get_model(model_name)
    jsonDicts.append(mod_best.params)

In [103]:
print(jsonDicts)

[{'model_id': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GBM_grid_1_AutoML_20190418_130009_model_1', 'type': 'Key<Model>', 'URL': '/3/Models/GBM_grid_1_AutoML_20190418_130009_model_1'}}, 'training_frame': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'automl_training_py_13_sid_a596', 'type': 'Key<Frame>', 'URL': '/3/Frames/automl_training_py_13_sid_a596'}}, 'validation_frame': {'default': None, 'actual': None}, 'nfolds': {'default': 0, 'actual': 5}, 'keep_cross_validation_models': {'default': True, 'actual': False}, 'keep_cross_validation_predictions': {'default': False, 'actual': True}, 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False}, 'score_each_iteration': {'default': False, 'actual': False}, 'score_tree_interval': {'default': 0, 'actual': 5}, 'fold_assignment': {'default': 'AUTO', 'actual

In [104]:
n=run_id+'EmpAccess_'+ str(run_time)+'_hy_parameter.json'
dict_to_json(jsonDicts,n)

In [105]:
jsonDictsActual = []
for m in model_set.iteritems():
    m,model_name = m
    mod_best = h2o.get_model(model_name)
    jsonDictsActual.append(mod_best.actual_params)

In [106]:
print(jsonDictsActual)

[<property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>]


### Generating metadata and storing in JSON

In [107]:
meta_data['end_time'] = time.time()

In [108]:
meta_data

{'project': 'KYUjJqIuc_EmpAccess_700',
 'run_time': 700,
 'run_id': 'KYUjJqIuc',
 'start_time_sec': 1555606780.2417612,
 'min_mem_size': 2,
 'balance': False,
 'balance_threshold': 0.2,
 'max_models': None,
 'scale': False,
 'model_path': None,
 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project',
 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv',
 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\KYUjJqIuc_EmpAccess_700',
 'nthreads': 1,
 'analysis': 0,
 'end_time_sec': 1555606780.2417612,
 'Problem_type': 'Classification',
 'Total Records': 327690,
 'Target': 'ACTION',
 'X': ['RESOURCE',
  'MGR_ID',
  'ROLE_ROLLUP_1',
  'ROLE_ROLLUP_2',
  'ROLE_DEPTNAME',
  'ROLE_TITLE',
  'ROLE_FAMILY_DESC',
  'ROLE_FAMILY',
  'ROLE_CODE'],
 'levels': [['0', '1']],
 'variables': {'ACTION': 'enum',
  'RESOURCE': 'int',
  'MGR_ID': 'int',
  'ROLE_ROLLUP_1': 'int',
  'ROLE_ROLLUP_2': 'int',
  'ROLE_DEPTNAME'

In [109]:
n=run_id+'EmpAccess_' + str(run_time) + 'meta_data.json'
dict_to_json(meta_data,n)

#### Shutting down H2O

In [110]:
# Clean up
os.chdir(server_path)

In [111]:
h2o.cluster().shutdown()

H2O session _sid_a596 closed.


#### Link of data set

https://www.kaggle.com/c/amazon-employee-access-challenge/overview

# Conclusion

# Contribution

percentage ratio - 80: 20

Self - 80%

External - 20%

Urja - 40%

Prakruthi - 40%

# Citation

https://github.com/prabhuSub/Hyperparamter-Samples

https://github.com/nikbearbrown/CSYE_7245/tree/master/H2O



# License

Copyright (c) 2019 Urja Jain, Prakruthi Bagur Suryanarayanaprasad

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.